<a href="https://colab.research.google.com/github/izu-hiro/RAG-Pageindex/blob/main/prototipo_rag/quary.teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Definindo a LLM

In [7]:
from openai import AsyncOpenAI
from getpass import getpass

openrouterKey = getpass("Insira a sua chave API: ")

async def call_llm(prompt, model="deepseek/deepseek-chat-v3.1:free", temperature=0):
    client = AsyncOpenAI(base_url="https://openrouter.ai/api/v1", api_key=openrouterKey)
    response = await client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content.strip()

Insira a sua chave API: ··········


Contexto utilizado na geração da resposta

In [11]:
from pypdf import PdfWriter
from os import listdir
from os.path import isfile, join

documentos = ['documentos/'+f for f in listdir('documentos') if isfile(join('documentos', f))]

merger = PdfWriter()
for documento in documentos:
    merger.append(documento)
merger.write("documentoUnido.pdf")
merger.close()

relevant_content = '''Manual Explicativo do Processo Seletivo Vestibular – 1º Semestre de 2026
1. Definição do Vestibular

O Vestibular das Faculdades de Tecnologia do Estado de São Paulo (Fatecs) é o processo seletivo utilizado para ingresso nos cursos superiores de tecnologia oferecidos pelas unidades do CEETEPS. As provas serão realizadas exclusivamente de forma presencial.

2. Público-Alvo

O Processo Seletivo Vestibular, referente ao 1º semestre de 2026, destina-se a candidatos que tenham concluído o Ensino Médio até a data da matrícula, conforme os seguintes critérios:

Ensino Médio regular concluído em 2025 ou anteriormente: é necessário apresentar Histórico Escolar com Certificado de Conclusão.

Ensino Médio concluído por meio de exames de certificação (ENEM, ENCCEJA e afins): é exigida a certificação oficial expedida por órgão competente.

Educação de Jovens e Adultos (EJA): para os candidatos que concluírem em 2025, é obrigatória a apresentação do Histórico Escolar com Certificado de Conclusão.

3. Requisitos de Idade

Apenas para o Curso Superior de Tecnologia em Radiologia há exigência etária. O estudante deverá ter no mínimo 18 anos para cursar as disciplinas práticas de laboratório, a partir do 4º semestre, em conformidade com o Parecer CNE/CEB 19/2007 e demais legislações vigentes.

4. Cronograma do Processo Seletivo

O cronograma oficial do Vestibular Fatec – 1º Semestre de 2026 é o seguinte:

15/09/2025 a 07/11/2025 (até às 15h): Período de inscrições no site vestibular.fatec.sp.gov.br
.

09/12/2025, a partir das 15h: Divulgação dos locais de prova.

14/12/2025, às 13h: Realização do exame.

17/12/2025, a partir das 15h: Divulgação do gabarito oficial.

19/01/2026, a partir das 15h: Publicação da lista de classificação geral e da 1ª chamada.

20 a 22/01/2026: Matrícula da 1ª chamada (envio da documentação pelo sistema remoto).

26/01/2026, a partir das 15h: Divulgação da 2ª chamada.

27 a 29/01/2026: Matrícula da 2ª chamada (envio da documentação pelo sistema remoto).

Cabe exclusivamente ao candidato acompanhar os prazos e publicações referentes a inscrições, provas, gabaritos, classificações e matrículas. Não serão aceitas documentações enviadas fora dos períodos estipulados.

5. Organização do Processo

O Vestibular Fatec 1SEM26 é organizado pela Fundação de Apoio à Tecnologia (FAT), entidade responsável pela operacionalização de mais de 30 edições anteriores. A FAT utiliza sistema informatizado proprietário que assegura segurança, integridade, escalabilidade e transparência durante todas as etapas.

6. Inscrições

As inscrições deverão ser realizadas entre 15/09/2025 e 07/11/2025 (até às 15h), exclusivamente pelo site oficial vestibular.fatec.sp.gov.br
. O candidato deverá acessar a Área do Candidato utilizando seu CPF e a senha cadastrada.

O formulário eletrônico de inscrição deverá ser preenchido integralmente, contemplando dados pessoais, questionário socioeconômico, indicação de cursos, opção pelo sistema de pontuação acrescida (quando aplicável), além da possibilidade de inclusão do número de inscrição do ENEM (anos 2023, 2024 ou 2025).

Ao final do preenchimento, o candidato deverá emitir o boleto e efetuar o pagamento da taxa de inscrição no valor de R$ 47,00, dentro do prazo estipulado, seja em agência bancária, internet banking ou via cartão de crédito (ferramenta Getnet).

O não pagamento dentro do prazo resultará na não efetivação da inscrição. O valor pago não será devolvido em nenhuma hipótese, inclusive em caso de desistência ou duplicidade de inscrição.

Candidatos que não possuam CPF deverão obtê-lo junto à Receita Federal, pois este é requisito obrigatório para inscrição.

Será permitida a realização de até três inscrições por candidato, mediante pagamento da taxa correspondente a cada inscrição.

7. Isenção e Redução da Taxa

Os candidatos que solicitarem isenção ou redução da taxa deverão verificar o resultado no site oficial a partir de 30/09/2025. As situações possíveis são:

Isenção integral aprovada: inscrição automaticamente confirmada.

Redução de 50% aprovada: emissão de boleto no valor de R$ 23,50 e pagamento até 07/11/2025.

Solicitação recusada: pagamento do valor integral (R$ 47,00).

O benefício será válido para apenas uma inscrição. Caso o candidato deseje se inscrever em mais de um curso, deverá pagar integralmente pelas demais inscrições.

8. Pontuação Acrescida

Conforme Decreto Estadual nº 49.602/2005 e Deliberação CEETEPS nº 08/2007, será atribuída pontuação adicional à nota final obtida no processo seletivo, nos seguintes termos:

3% para candidatos que se declararem afrodescendentes;

10% para candidatos que tenham cursado integralmente o Ensino Médio em escolas públicas brasileiras;

13% para candidatos que atendam cumulativamente aos dois critérios anteriores.

A comprovação da escolaridade pública deverá ser feita no ato da matrícula, por meio de histórico escolar ou declaração em papel timbrado da instituição. Informações falsas acarretarão cancelamento da matrícula, sem possibilidade de reclassificação.

9. Condições Especiais

Candidatos com deficiência participarão do processo em igualdade de condições com os demais, sem previsão de cotas específicas.'''

Geração da resposta

In [9]:
perguntas = [
    "Qual é a idade mínima para poder ingressar na fatec?",
    "Quanto custa pra estudar numa instituicao do centro paula souza?",
    "Como q funciona o processo de inscrição no vestibulinho da etec?",
    "Fatec tem programa de mestrado? se sim, qnd começam as aulas do mestrado de 2025?"
]

for query in perguntas:
    answer_prompt = f"""
Responda à pergunta com base apenas no contexto fornecido abaixo.

- Interprete a pergunta em **português**, mesmo que ela contenha:
  - Gírias ou linguagem informal
  - Erros de digitação ou ortografia
  - Abreviações ou escrita de forma não convencional
  - Estruturas de frase incompletas ou confusas
- Se houver ambiguidade, faça a melhor interpretação possível considerando o contexto.
- Mantenha a resposta **clara, completa e em português correto**.
- Não invente informações que não estejam no contexto.
- Use exemplos ou explicações adicionais, se necessário, para garantir que a resposta seja compreendida.


    Pergunta do usuário: {query}
    Contexto: {relevant_content}

    Resposta:
    """

    answer = await call_llm(answer_prompt)

    print("Pergunta original:\n", query)
    print("\nResposta gerada (PT):\n", answer)


Pergunta original:
 Qual é a idade mínima para poder ingressar na fatec?

Resposta gerada (PT):
 Com base no contexto fornecido, **não há uma idade mínima geral para ingressar na Fatec**.

A exigência de idade é uma exceção e aplica-se apenas a um curso específico:
*   **Para o curso de Tecnologia em Radiologia**, o candidato deve ter no mínimo 18 anos de idade. Esta exigência é necessária para cursar as disciplinas práticas de laboratório a partir do 4º semestre.

Para todos os demais cursos, o requisito principal é acadêmico: o candidato deve ter concluído o Ensino Médio até a data da matrícula.
Pergunta original:
 Quanto custa pra estudar numa instituicao do centro paula souza?

Resposta gerada (PT):
 Com base no contexto fornecido, o custo para se inscrever no processo seletivo (Vestibular) das Fatecs do Centro Paula Souza é de **R$ 47,00**.

Esta é a taxa de inscrição para participar do vestibular do 1º semestre de 2026. O contexto não menciona o valor da mensalidade para estudar 